In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-30 07:44:41--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.1s    

2021-01-30 07:44:42 (6.50 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [ ]:
from pyspark import SparkFiles
url = ""
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

In [ ]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://britneydatabucket.s3.us-east-2.amazonaws.com/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

# Show DataFrame
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [ ]:
df.count()

4751577

In [ ]:
# Drop duplicates and incomplete rows
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

4751577
4750852
4750852


In [ ]:
# Examine the schema
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



### Create Vine DataFrames to match tables

In [ ]:
# Create the vine_table. DataFrame
vine_df = df1.select(["review_id", "star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1005BINTGKCVE|          5|            0|          0|   N|                Y|
|R10067SN6FSII1|          4|            0|          0|   N|                N|
|R100NGDZ6CAL8E|          1|            2|          2|   N|                N|
|R100OLMAO039IL|          3|            2|          2|   N|                N|
| R100OMDUXE5VZ|          4|            0|          0|   N|                Y|
|R100RJOC7TCXN6|          5|            2|          2|   N|                Y|
|R10182KW3O6486|          4|            5|          6|   N|                N|
|R101B8W76YQILD|          4|            5|          5|   N|                N|
|R101K97ELZLSDK|          4|            7|          9|   N|                N|
|R101MMM4IXONBO|          5|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [ ]:
# Configure settings for RDS
mode = "append"
#jdbc_url="jdbc:postgresql://<endpoint>:5432/<database name>"
jdbc_url="jdbc:postgresql://databritney.ccatdehwe7na.us-east-2.rds.amazonaws.com:5432/dataBritney"
config = {"user":"postgres", 
          "password": "Chicago2018!", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)

***Deliverable 2: Analyze the data and determine if the Vine reviews are biased.***

Our goal in this portion of the challenge is to determine if the Vine reviews are bias. The vine column is in boolean fashion and rows consist of "**Y" for "YES" and "N" for "NO"**. It is answering the question, "Was the review written as part of the Vine program?".

The analysis steps are:


*   We cropped the df.dropna() in the beginning, 
*   We started with the total Vine_df,
*   fliterd by total_votes >= 20, (vote count over 50% (vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5))
*   analyzed the five star % when vine == Y (paid)
*   analyzed the five star % when  vine == N (not paid)



In [ ]:
# vine_analysis - let's check the vine_df table schema 
# vine_df = df1.select(["review_id", "star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)



In [ ]:
#vote count
vote_count = df.filter("total_votes >= 20")

In [ ]:
#create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
new_table = vote_count.filter(vote_count["helpful_votes"]/vote_count["total_votes"]>=0.5)
new_table.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14231359|R12JK6J668AS5Z|B00ZXF9PHG|     990620497|Batman: Arkham Kn...|           Music|          2|           20|         21|   N|                Y|Trashy CD-R Produ...|I really like the...| 2015-07-18|
|         US|   42556078|R3M7195WO9ZSXK|B00UN9PP44|     798531319|Sticky Fingers (2...|           Music|          5|    

In [ ]:
#reate a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'
new_table.filter(new_table["vine"] == "Y").show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   49197984|R1JZ0JAPW83WFS|B001L2I27O|     995259710|          Troubadour|           Music|          4|           54|         58|   Y|                N|An old white lady...|I try to be hip, ...| 2009-05-21|
|         US|   53081008| R4V3ICFDTIDIF|B001SMC91M|     609338227|Tell 'Em What You...|           Music|          4|    

In [ ]:
#Create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'
new_table.filter(new_table["vine"] == "N").show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   14231359|R12JK6J668AS5Z|B00ZXF9PHG|     990620497|Batman: Arkham Kn...|           Music|          2|           20|         21|   N|                Y|Trashy CD-R Produ...|I really like the...| 2015-07-18|
|         US|   42556078|R3M7195WO9ZSXK|B00UN9PP44|     798531319|Sticky Fingers (2...|           Music|          5|    

In [ ]:
# cout the vine_df
vine_df.count()

4750852

In [ ]:
# number of five star reviews
five_star = new_table.filter(new_table["star_rating"]== 5)
five_star.cache()

DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]

In [ ]:
# cout the five star
five_star.count()

67568

In [ ]:
#total reviews
new_table.count()

105969

In [ ]:
#total reviews
new_table.count()

105969

In [ ]:
#percentage of paid reviews
five_star.filter(five_star["verified_purchase"] == "Y").count()/new_table.filter(new_table["verified_purchase"]== "Y").count()

0.6676168119830092

In [ ]:
#percentage of unpaid reviews
five_star.filter(five_star["verified_purchase"] == "N").count()/new_table.filter(new_table["verified_purchase"]== "N").count()

0.631526959365101

**determining if the Vine reviews are biased**

Here is the final summary:

*   Total rows of Vine_df: 4750852
*   Total Reviews (vote count> 50% and total_votes > 20):105969
*   Total five star_counts: 67568
*   analyzed the five star % when vine == Y (Paid):0.6676168119830092
*   analyzed the five star % when vine == N (not paid): 0.631526959365101

In this summary, when vine is Yes (means paid), the five star rate is slightly higher, and seems biased 

(0.6676168119830092 - 0.631526959365101 = 0.0360 )

Although, it is possible that there could have other factors, or it is a simply coinsident, and my conclusion is it maybe be biased, but not a gerat deal and certain.
Thank you! It is a greatly fun project.

---


